In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 6.0MB/s 
     |████████████████████████████████| 2.9MB 35.7MB/s 
     |████████████████████████████████| 890kB 38.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c18740705eb57b2f5ce7e64a4265d7e8e32451429d1dbd01ba5f9187b2c0f026
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!git clone https://github.com/NLP-kr/tensorflow-ml-nlp-tf2.git

Cloning into 'tensorflow-ml-nlp-tf2'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 1656 (delta 11), reused 7 (delta 3), pack-reused 1631
Receiving objects: 100% (1656/1656), 201.15 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (1001/1001), done.
Checking out files: 100% (88/88), done.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import re
%matplotlib inline

In [ ]:
import os
os.chdir('./tensorflow-ml-nlp-tf2/')
!ls

1.Intro		       7.PRETRAIN_METHOD	 exec_jupyter_for_gpu.sh
2.NLP_PREP	       build_jupyter_for_cpu.sh  LICENSE
3.NLP_INTRO	       build_jupyter_for_gpu.sh  main.png
4.TEXT_CLASSIFICATION  Dockerfile_cpu		 README.md
5.TEXT_SIM	       Dockerfile_gpu		 requirements_dkr.txt
6.CHATBOT	       exec_jupyter_for_cpu.sh	 requirements.txt


In [ ]:
DATA_IN_PATH = "./4.TEXT_CLASSIFICATION/data_in/"
DATA_OUT_PATH = "./data_out/"
print("file size: ")
for file in os.listdir(DATA_IN_PATH):
    if 'txt' in file:
        print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH+file)/ 1000000, 2)) + 'MB')

file size: 
ratings.txt                   19.52MB
ratings_train.txt             14.63MB
ratings_test.txt              4.89MB


In [ ]:
from transformers import *
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
def bert_tokenizer(sent, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True, # add [CLS] and [SEP]
        max_length = MAX_LEN,
        pad_to_max_length = True,
        return_attention_mask = True
    )

    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # and its attention mask (simply differentiates padding from non-padding)
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences

    return input_id, attention_mask, token_type_id

In [ ]:
bert_tokenizer("교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정", 10)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


([101, 8907, 12092, 22333, 110148, 17196, 118922, 119, 119, 102],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

네이버 영화 리뷰 데이터 전처리

In [ ]:
# data load
train_data = pd.read_csv(DATA_IN_PATH + 'ratings_train.txt', header=0 , delimiter = '\t', quoting = 3)
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# special tokens
print(tokenizer.all_special_tokens, "\n", tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] 
 [100, 102, 0, 101, 103]


In [ ]:
# test tokenizer
kor_encode = tokenizer.encode("안녕하세요, 반갑습니다")
eng_encode = tokenizer.encode("Hello world")

kor_decode = tokenizer.decode(kor_encode)
eng_decode = tokenizer.decode(eng_encode)

print("encode")
print(kor_encode)
print(eng_encode)
print("\ndecode")
print(kor_decode)
print(eng_decode)

encode
[101, 9521, 118741, 35506, 24982, 48549, 117, 9321, 118610, 119081, 48345, 102]
[101, 31178, 11356, 102]

decode
[CLS] 안녕하세요, 반갑습니다 [SEP]
[CLS] Hello world [SEP]


In [ ]:
train_data["label"]

0         0
1         1
2         0
3         0
4         1
         ..
149995    0
149996    1
149997    0
149998    1
149999    0
Name: label, Length: 150000, dtype: int64

In [ ]:
input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []
MAX_LEN=39

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
    return sent_clean

for train_sent, train_label in zip(train_data["document"], train_data["label"]):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent), MAX_LEN)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)
    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_movie_input_ids = np.array(input_ids, dtype=int)
train_movie_attention_masks = np.array(attention_masks, dtype=int)
train_movie_type_ids = np.array(token_type_ids, dtype=int)
train_movie_inputs = (train_movie_input_ids, train_movie_attention_masks, train_movie_type_ids)

train_data_labels = np.asarray(train_data_labels, dtype=np.int32) # 정답 토크나이징 리스트

print(f"# sents: {len(train_movie_input_ids)}, # labels: {len(train_data_labels)}")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


expected string or bytes-like object
nan
expected string or bytes-like object
nan
expected string or bytes-like object
nan
expected string or bytes-like object
nan
expected string or bytes-like object
nan
# sents: 149995, # labels: 149995


In [ ]:
num=20
input_id = train_movie_input_ids[num]
attention_mask = train_movie_attention_masks[num]
token_type_id = train_movie_type_ids[num]

print(input_id)
print(attention_mask)
print(token_type_id)
print(tokenizer.decode(input_id))

[   101   8982  49543   9491  28188  11102   9153  12092  13767   9120
   8924 118729   9953  24017  10739   9428  24017  11882   9025  16985
  49742  42428  11018   9666  14423   9519 108578    102      0      0
      0      0      0      0      0      0      0      0      0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
[CLS] 나름 심오한 뜻도 있는 듯 그냥 학생이 선생과 놀아나는 영화는 절대 아님 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


네이버 영화 리뷰 모델 학습

In [ ]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range),
                                                name="classifier")
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attention)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits 

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased', 
                                  dir_path='bert-ckpt',
                                  num_class=2)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

cls_model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=[metric])

In [ ]:
model_name = "tf2_bert_naver_movie"

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weight.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print(f"{checkpoint_dir} -- Folder already exists \n")
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print(f"{checkpoint_dir} -- Folder create complete \n")

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weight_only=True
)


BATCH_SIZE = 32
NUM_EPOCHS = 3
VALID_SPLIT = 0.2

history = cls_model.fit(train_movie_inputs,
                    train_data_labels,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=VALID_SPLIT,
                    callbacks=[earlystop_callback, cp_callback])

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


./data_out/tf2_bert_naver_movie -- Folder already exists 

Epoch 1/3


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


3750/3750 [==============================] - ETA: 0s - loss: 0.7236 - accuracy: 0.5018

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


3750/3750 [==============================] - 2168s 575ms/step - loss: 0.7236 - accuracy: 0.5018 - val_loss: 0.7149 - val_accuracy: 0.4973

Epoch 00001: val_accuracy improved from -inf to 0.49728, saving model to ./data_out/tf2_bert_naver_movie/weight.h5
Epoch 2/3
  12/3750 [..............................] - ETA: 33:32 - loss: 0.7373 - accuracy: 0.4526